<a href="https://colab.research.google.com/github/Bennybeams/DataExplorationPython/blob/main/Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets

Explore data with Python:
1. Download the dataset
2. Prepare and clean the data
3. Analysis and visualization
4. Questions and answers

**1. Download the dataset**

We will use in this course a dataset which gives the price of some material in agriculture. It's a kaggle dataset named agricultural-raw-material-prices-19902020

To use it you will need a kaggle account, then go to your account, API, and create a new API token. Use the username and key from the downloaded json.

od.download function download the dataset as csv to the root of the working directory.

In [ ]:
dataset_url = 'https://www.kaggle.com/kianwee/agricultural-raw-material-prices-19902020'
import opendatasets as od
od.download(dataset_url)

**2. Data preparation and cleaning**

We will use numpy and pandas package to explore our dataset

In [6]:
import numpy as np
import pandas as pd

Now we store our dataset into a dataframe

In [7]:
df=pd.read_csv('./agricultural-raw-material-prices-19902020/agricultural_raw_material.csv')

Let's see the size of the dataset, and how much null values we have

In [ ]:
df.isnull()

In [ ]:
print(df.shape)
#Checking Null Values of each column
df.isnull().sum()

Now let's see a resume of the table

In [ ]:
df.head()

**Conclusion**

We have some null values, and numeric data as string (the % ones). To use it in analyses we need to convert it into numeric.

How to deal with these values ? We will use nothing instead of non numeric character such as %, comma or -, and will use Nan instead of ''.

In [ ]:
# Replacing %, "," and "-"
df = df.replace('%', '')
df = df.replace(',', '')
df = df.replace('-', '')
df = df.replace('', np.nan)
#df = df.replace('MAY90', np.nan)
# Dropping rows with NaN values
df = df.dropna()
# Check to see if all NaN values are resolved
df.isnull().sum()

In [ ]:
df

Now let's convert all these values into numeric !

In [ ]:
# Create a list with all the columns names but not the first one (which is not numeric)
lst = list(df.columns) 
del lst[0]
# Convert all the column which the name is in this list as float
df[lst] = df[lst].astype("float")
# Is it correct ?
df.dtypes

Now let's format the first column Date, and set it as index for this dataset

In [11]:
df.Month  = pd.to_datetime(df.Month.str.upper(), format='%b%y', yearfirst=False)
# Indexing month
df = df.set_index('Month')

Month is not a column anymore but replace index 0,1,2,3,etc...

In [ ]:
df.head()

Dataset is now clean and can be explored

**3. Analysis and Visualization**

First we need to import some libraries : Seaborn and Matplotlib

In [13]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (9, 5)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

Let's now study the correlation between the prices.
- First we'll do a correlation matrix between Prices columns (excluding variation columns)
- Then we'll present the values in a Heatmap.

In [14]:
#raw-materials list

raw_data=list(filter(lambda x : '%' not in x , lst)) # lambda : for any x into lst if x do not contains % then include x
raw_data

['Coarse wool Price',
 'Copra Price',
 'Cotton Price',
 'Fine wool Price',
 'Hard log Price',
 'Hard sawnwood Price',
 'Hide Price',
 'Plywood Price',
 'Rubber Price',
 'Softlog Price',
 'Soft sawnwood Price',
 'Wood pulp Price']

In [ ]:
#getting the correlation matrix
corrmat = df[raw_data].corr()
corrmat

In [ ]:
#setting the size of plot
plt.figure(figsize = (12, 9))
#masking the upper traingle part since matrix is symmetric(repetitive)
sns.heatmap(corrmat, vmax = 1, annot = True)
plt.show() 

As we don't need to have all the values (same couple have the same correlation and as an item will have a correlation of 1 with itself) we would like to apply a mask to remove unecessary values.

In [ ]:
mask = np.triu(np.ones_like(corrmat)) #create a matrix with 1 for unwanted values
print(mask)
plt.figure(figsize = (12, 9))
sns.heatmap(corrmat, vmax = 1,mask=mask, annot = True) #apply the mask to remove unwanted values from the plot
plt.show()

We can say that there is some correlations between the prices. Per example, Fine wool Price is correlated with Coarse wool Price (which looks logical).
Now let's see the correlation between the evolutions

In [ ]:
#raw-materials list
changelist=list(filter(lambda x : '%' in x , lst)) #lambda : for any x into lst if x do contains % then include x
changelist

In [ ]:
#getting the correlation matrix
corrmat = df[changelist].corr()
corrmat

In [ ]:
mask = np.triu(np.ones_like(corrmat)) #create a matrix with 1 for unwanted values
plt.figure(figsize=(30,15))
sns.heatmap(corrmat, vmax = 1,mask=mask, annot = True) #apply the mask to remove unwanted values from the plot
plt.show()

**Conclusion**

Where prices may be correlated, we can see that price evolution are not really.

As refresh :
- Coefficient close to 0 means that there is no correlation
- Coefficient close to 0.476 means that there is a moderate correlation
- Coefficient close to 1 means that there is a strong correlation
- Coefficient close to -1 means that there is a strong negative correlation (if one grows, the other one decrease) 

Let's now plot information about the wool.
We want have a line plot for each column containing 'wool' string. This may be done in only one line in Python.

In [ ]:
productList = list(filter(lambda x : 'wool' in x,lst))
productPlot = df[productList].plot(figsize=(30,15), subplots=True, linewidth=1)

We could also want to compare the prices between wool and coton per example (without the evolution)

In [ ]:
productList = list(filter(lambda x : ('Coarse wool' in x or 'Cotton' in x) and '%' not in x,lst))
productPlot = df[productList].plot(figsize=(30,15), subplots=True, linewidth=1)

**4. Questions and answers**

**4.1 - What is the usual price change for each raw material ? What is the more and the less stable price ?**

This can be answered with a density plot of the price change % for each raw material.

In [ ]:
changelist=list(filter(lambda x : '%' in x,lst))
distribplot=df[changelist].plot.density(figsize=(30,15), linewidth=1)
distribplot.set_xlim(-20,20)

**Answer**

- Regular variations are between around -2% and +6%
- The more stable price is for Hard sawnwood (narrowest distribution)
- The less stable price is for Wood pulp (widest distribution)

**4.2 - What material have the lowest price over the years**

To see the evolution, the first approach is to make a line plot with all the materials and compare.

In [ ]:
priceList=list(filter(lambda x : '%' not in x,lst))
pricePlot = df[priceList].plot(figsize=(30,15), linewidth=1)
plt.suptitle("Raw-Materials price comparision")

It looks like cotton and rubber price are very low as they are stuck at the bottom of the plot. Let's confirm that by plotting each chart separately in a chart table.

In [ ]:
plt.figure(figsize=(20, 10))
for i in range(len(priceList)):
    plt.subplot(4,3,i+1)
    plt.subplots_adjust( hspace=1 ,wspace=0.5)
    plt.title(priceList[i])
    plt.plot(df[priceList[i]])
    plt.xticks(rotation=45)
plt.suptitle("Raw-Materials price comparision")

It's confirmed : with a price at around 5 at maximum rubber and cotton are the cheapest materials.